# Example on how to create an adaptive fast 2D-scan, using the Basel Precision Instruments LNHR DAC II Telnet module

Copyright (c) Basel Precision Instruments GmbH (2024)

...............................................................................................................


The LNHR DAC II is by default equipped with all the necessary functions to perform a fast adaptive 2D-scan. In a typical fast adaptive 2D-scan scenario, two outputs of the LNHR DAC II are used to create the x-axis and y-axis signals and a third output is used to create a trigger signal, that can be used to trigger the data acquisition to measure evry point in the 2D-scan.

The faster y-axis signal is created using an AWG, the slower x-axis signal is created using a Ramp/Step-Generator of the LNHR DAC II. The x-axis gets automatically updated internally, after each fast y-axis cycle. The trigger signal is created using a secon AWG of the LNHR DAC II.

## 1 - imports and setup
For this example the Basel Precision Instruments LNHR DAC II Telnet driver is used (available on Github).

In [30]:
from telnet import LNHRDAC

# create an instance of the LNHR DAC device
DAC = LNHRDAC("192.168.0.5", 23)

Connected to DAC (192.168.0.5) successfully. The current status of all channels is shown below (channel 1; channel 2; ... ):
OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF



## 2 - define parameters for the 2D-scan
To simplify the setup process of the 2D-scan, a few parameters are defined here, from which the actual DAC parameters are derived from:

- **x_steps**: number of steps on the slower x-axis (int)
- **x_start_voltage**: voltage at which the slower x-axis starts a sweep (float)
- **x_stop-voltage**: voltage at which the slower x-axis stops a sweep (float)
- **y_steps**: number of steps on the faster y-axis (int)
- **y_start_voltage**: voltage at which the faster y-axis starts a sweep (float)
- **y_stop-voltage**: voltage at which the faster y-axis stops a sweep (float)
- **acquisition_delay**: time the data acquisition needs to measure the signal in ms (milli-seconds) or "duration of a point" (float)
- **adaptive_shift**: voltage shift after each fast sweep of the y-axis


In [65]:
x_steps = 10
x_start_voltage = 0.0
x_stop_voltage = 1.0
y_steps = 10
y_start_voltage = 0.0
y_stop_voltage = 1.0
acquisition_delay = 10
adaptive_shift = 0.1

## 3 - choose DAC channels for x- and y-axis
The output for the x- and y-axis can be selected freely, as long as both outputs are on the same DAC-Board (channels 1 - 12 or 13 - 24). For this example the AWG-A and RAMP-A generators are used.

In [66]:
# define the DAC channels which will be used as outputs
output_x = 1
output_y = 2

# check availability of the AWG-A and RAMP-A, set auxiliary variables
if DAC.expect_query_answer("c awg-a ava?", "1") and DAC.expect_query_answer("c rmp-a ava?", "1"):
    pass
else:
    raise Exception("AWG resources not available")

awg_output = "a"
memory = 0
board = "ab"

## 4 - setup x-axis
The LNHR DAC II must be configured to automatically update the slower x-axis signal using the before chosen ramp generator.

In [67]:
# calculate the internally used parameter
ramp_time = 0.005 * (x_steps + 1)

DAC.send_command(f"c rmp-{awg_output} ch {output_x}") # ramp selected DAC-channel
DAC.send_command(f"c rmp-{awg_output} stav {x_start_voltage: .6f}") # ramp start voltage
DAC.send_command(f"c rmp-{awg_output} stov {x_stop_voltage: .6f}") # ramp stop voltage
DAC.send_command(f"c rmp-{awg_output} rt {ramp_time: .3f}") # ramp time
DAC.send_command(f"c rmp-{awg_output} rs 0") # ramp shape
DAC.send_command(f"c rmp-{awg_output} cs 1") # ramp cycles set
DAC.send_command(f"c rmp-{awg_output} step 1") # ramp/step selection

## 5 - setup y-axis
The y-axis signal is generated using an AWG of the LNHR DAC II.

### 5.1 - Configuring the AWG
The before chosen AWG-A is configured to the correct update rate. It must be checked, that the minimum AWG duration is at least 6 ms.

In [68]:
# calculate the internally used parameters
clock_period = int(acquisition_delay * 1000)
frequency = 1.0 / (y_steps * (0.000001 * clock_period))
amplitude = y_stop_voltage - y_start_voltage
offset = y_start_voltage

# check minimum AWG duration
if (1.0 / frequency) < 0.006:
    raise Exception("Y axis: clock period too short or not enough steps")

# configure AWG
DAC.send_command(f"c awg-{awg_output} ch {output_y}") # awg selected DAC-channel
DAC.send_command(f"c awg-{awg_output} cs 1") # awg cycles set
DAC.send_command(f"c awg-{awg_output} tm 0") # awg external trigger
DAC.send_command(f"c awg-{board} cp {clock_period}") # clock period
DAC.send_command(f"c swg aclk 0") # adaptive clock period

### 5.2 - standard waveform generation
The internal Standard Waveform Generator (SWG) is used to create the fast y-axis signal. It is configured to generate a ramp. It can also be used to generate simple standard waveforms for other applications.

In [69]:
DAC.send_command(f"c swg mode 0") # swg mode (generate new/ use old)
DAC.send_command(f"c swg wf 3") # swg function
DAC.send_command(f"c swg df {frequency: .3f}") # swg frequency
DAC.send_command(f"c swg amp {amplitude: .6f}") #swg amplitude
DAC.send_command(f"c swg dcv {offset: .6f}") # swg offset
DAC.send_command(f"c swg pha 0.0000") # swg phase
DAC.send_command(f"c swg wmem {memory}") # selected wave memory
DAC.send_command(f"c swg wfun 2") # selected wave memory operation

## 5.3 - write the generated waveform to the AWG
The before created waveform must be saved to the wave memory before it can be transferred to the AWG memory.

In [70]:
DAC.send_command(f"c wav-{awg_output} clr") # clear wave memory
DAC.send_command(f"c swg apply") # apply swg to wave memory
while DAC.expect_query_answer(f"c wav-{awg_output} busy?", "1"): # check if writing to memory is done
    pass
last_mem_adr = DAC.send_query(f"c wav-{awg_output} ms?") # get wave memory address
last_mem_adr = int(last_mem_adr)
DAC.send_command(f"wav-{awg_output} {last_mem_adr: x} {y_start_voltage: .6f}") # set last step to start value
DAC.send_command(f"c wav-{awg_output} write") # write wave memory to awg memory
while DAC.expect_query_answer(f"c wav-{awg_output} busy?", "1"): # check if writing to memory is done
    pass

## 6 - setup the adaptive shift
A linear adaptive shift is executed when the parameter `adaptive_shift` is not zero. After each fast y-axis cycle, the start and stop voltage for the next cycle are shifted by the defined voltage. 

In [71]:
adaptive_scan = 1 if adaptive_shift > 0.0 else 0

DAC.send_command(f"c awg-{awg_output} as 1") # normal/ auto start awg
DAC.send_command(f"c awg-{awg_output} rld {adaptive_scan}") # keep/reload awg memory
DAC.send_command(f"c awg-{awg_output} ap {adaptive_scan}") #apply/skip polynomial
DAC.send_command(f"c awg-{awg_output} shiv {adaptive_shift: .6f}") # adaptive shift voltage

## 7 - setup trigger for data acquisition
To set the trigger output up, the same steps as for creating the y-axis signal are repeated, since both use an AWG of the LNHR DAC II. Instead of a ramp this time a rectangular signal is created. 

**Caution:** The trigger needs a different time base (clock period) than both axes. Therefore it is not possible to have the trigger output on the same DAC-Board as the axes. Additionally it is necessary to connect the `SYNC OUT AWG` signal from the fast y-axis (in this example `SYNC OUT AWG A`) to the `TRIG IN AWG` input of the data acquisition trigger signal (in this example `TRIG IN AWG C`). Those are physical connections on the backside of the LNHR DAC II. This synchronizes the trigger signal for the data aqcuisition to the y-axis signal.

In [72]:
# define the DAC channel which will be used as output
output_daq_trigger = 13

# check availability of the AWG-C and RAMP-C, set auxiliary variables
if DAC.expect_query_answer("c awg-c ava?", "1") and DAC.expect_query_answer("c rmp-c ava?", "1"):
    awg_trigger = "c"
    memory = 2
    
# configure AWG
DAC.send_command(f"c awg-{awg_trigger} ch {output_daq_trigger}") # awg selected DAC-channel
DAC.send_command(f"c awg-{awg_trigger} cs {y_steps}") # awg cycles set
DAC.send_command(f"c awg-{awg_trigger} tm 1") # awg external trigger
DAC.send_command(f"c swg aclk 1") # adaptive clock period

# create rectangular signal using the SWG
DAC.send_command(f"c swg mode 0") # swg mode (generate new/ use old)
DAC.send_command(f"c swg wf 4") # swg function
DAC.send_command(f"c swg df {1/(clock_period*0.000001): .6f}") # swg frequency
DAC.send_command(f"c swg amp 2.500000") #swg amplitude
DAC.send_command(f"c swg dcv 2.500000") # swg offset
DAC.send_command(f"c swg pha 0.0000") # swg phase
DAC.send_command(f"c swg duc 50.000") # swg duty cycle
DAC.send_command(f"c swg wmem {memory}") # selected wave memory
DAC.send_command(f"c swg wfun 2") # selected wave memory operation

# write signal to wave memory and AWG memory afterwards
DAC.send_command(f"c wav-{awg_trigger} clr") # clear wave memory
DAC.send_command(f"c swg apply") # apply swg to wave memory
while DAC.expect_query_answer(f"c wav-{awg_trigger} busy?", "1"): # check if writing to memory is done
    pass
DAC.send_command(f"c wav-{awg_trigger} write") # write wave memory to awg memory
while DAC.expect_query_answer(f"c wav-{awg_trigger} busy?", "1"): # check if writing to memory is done
    pass

## 8 - prepare outputs and start 2D-scan
Before the 2D-scan can be started, the used outputs should be set to the assigned starting voltages. Additionally the Bandwidth should be set and the outputs should be turned on.

To start the 2D-scan, simply start the first cycle of the y-axis signal. If everything is configured correctly, the rest should happen fully automatic.

In [74]:
# set starting voltages
DAC.send_command(f"{output_x} {int((float(x_start_voltage) + 10.000000) * 838860.74): x}")
DAC.send_command(f"{output_y} {int((float(y_start_voltage) + 10.000000) * 838860.74): x}")

# set bandwidth
DAC.send_command(f"{output_x} hbw")
DAC.send_command(f"{output_y} hbw")
DAC.send_command(f"{output_daq_trigger} hbw")

# turn on outputs
DAC.send_command(f"{output_x} on")
DAC.send_command(f"{output_y} on")
DAC.send_command(f"{output_daq_trigger} on")

# start 2D-scan
DAC.send_command(f"c awg-{awg_output} start")

## 9 - restart or change parameters of the 2D-scan
It is recommended to turn all used outputs off and stop all AWGs and ramp generators before the 2D-scan is restarted or any parameters are changed.

In [60]:
#  turn off outputs
DAC.send_command(f"{output_x} off")
DAC.send_command(f"{output_y} off")
DAC.send_command(f"{output_daq_trigger} off")

# stop AWGs and RAMPs
DAC.send_command(f"c awg-{awg_output} stop")
DAC.send_command(f"c rmp-{awg_output} stop")
DAC.send_command(f"c awg-{awg_trigger} stop")